In [1]:
import os
import math
import numpy as np
from matplotlib import pyplot as plt
import time
import torch

testDir = "/home/mrim/quenot/irim/pytorch/dual_encoding_experiments/VisualSearchResults/msrvtt10k/test_results"

In [2]:
def tpca(a, q=65536):
    t0 = time.time()
    a = torch.tensor(a)
    m = a.mean(axis = 0)
    u, s, v = torch.pca_lowrank(a, q=min(a.size(1), q), center=True)
    print(time.time()-t0)
    return m.numpy(), u.numpy(), s.numpy(), v.numpy()

def loadEmbs(testDir, expName, runNum, embsName="embs.pth"):
    runDir = os.path.join(testDir, expName, "runs_%d/model_best" % runNum)
    embs = torch.load(os.path.join(runDir, embsName))
    return embs

def saveEmbs(embs, testDir, expName, runNum, embsName="embs_pca.pth"):
    runDir = os.path.join(testDir, expName,"runs_%d/model_best" % runNum)
    torch.save(embs, os.path.join(runDir, embsName))

def tpca(a, q=65536):
    t0 = time.time()
    m = a.mean(axis = 0)
    u, s, v = torch.pca_lowrank(a, q=min(a.size(1), q), center=True)
    print(time.time()-t0)
    return m, u, s, v

In [3]:
expName = "hybrid_1536_512_xin"
nr = 10
for n in range(nr):
    # print(n)
    embs = loadEmbs(testDir, expName, n)
    ncap = embs['cap_embs'].shape[0]
    nvid = embs['video_embs'].shape[0]
    vcl = torch.cat((torch.tensor(embs['cap_embs']), torch.tensor(embs['video_embs'])), dim=0)
    vcc = torch.cat((torch.tensor(embs['cap_tag_scores']), torch.tensor(embs['video_tag_scores'])), dim=0)
    ml, ul, sl, vl = tpca(vcl)
    mc, uc, sc, vc = tpca(vcc)
    rvcl = (torch.matmul(vcl-ml, vl))
    rvcc = (torch.matmul(vcc-mc, vc))
    rcapl, rvidl = rvcl[:ncap,:], rvcl[ncap:,:]
    rcapc, rvidc = rvcc[:ncap,:], rvcc[ncap:,:]
    embs['cap_embs'], embs['video_embs'] = rcapl.numpy(), rvidl.numpy()
    embs['cap_tag_scores'], embs['video_tag_scores'] = rcapc.numpy(), rvidc.numpy()
    saveEmbs(embs, testDir, expName, n)

21.057140111923218
6.217867851257324
18.99947214126587
5.9528820514678955
21.471531629562378
5.908579111099243
17.66382598876953
6.103328227996826
17.76872491836548
6.3718554973602295
20.29757595062256
5.832735061645508
20.626039266586304
5.76411247253418
21.67258906364441
5.553995370864868
20.719910144805908
5.605205535888672
20.31920576095581
5.794013023376465
